In [ ]:
import openpyxl

# Load the Excel workbook from the specified file path
wb = openpyxl.load_workbook("C:/Users/sample.xlsx")

# Select the first sheet in the workbook
sh = wb['Sheet1']

# Create an empty list to store the data from the Excel sheet
sheet_cells = []

# Iterate through all the rows in the selected sheet
for rows in sh.iter_rows():
    row_cells = []

    # Iterate through all the cells in the current row
    for cell in rows:
        row_cells.append(cell.value)

    # Convert the row data into a tuple and add it to the sheet_cells list
    sheet_cells.append(tuple(row_cells))

# Loop through the data extracted from the Excel sheet
for index, tuple in enumerate(sheet_cells):
    # Extract values from the tuple (assuming it contains store code, latitude, and longitude)
    store_code = tuple[0]
    lat = tuple[1]
    lon = tuple[2]

    # Print the extracted values for each row
    print(store_code, lat, lon)


In [2]:
import time
import math
import googlemaps # pip install googlemaps
import pandas as pd # pip install pandas
import openpyxl


In [3]:
# Function to convert kilometers to meters
def km_to_meters(km):
    try:
        return km * 1000
    except:
        return 0

# Your Google Maps API Key
API_KEY = 'YOUR_API_KEY'
map_client = googlemaps.Client(API_KEY)

In [5]:
# List of keywords representing types of businesses or points of interest
search_string = ['mosque', 'school', 'store', 'bank', 'restaurant', 'clothing_store', 'health', 'mosque',
                 'hospital', 'car_dealer', 'finance', 'electronics_store', 'university', 'home_goods_store']

# Convert search radius to meters (100 meters in this case)
distance = km_to_meters(0.1)

# Initialize an empty list to store business data
business_list = []

# Assuming sheet_cells is a list of tuples with store code, latitude, and longitude
for index, tuple in enumerate(sheet_cells):
    store_code = tuple[0]
    lat = tuple[1]
    lon = tuple[2]

    # Initialize an empty list to store business data for this location
    business_list = []

    # Loop through each search keyword
    for string in search_string:
        coordinates = (lat, lon)

        # Perform a nearby places search using Google Maps Places API
        response = map_client.places_nearby(
            location=coordinates,
            keyword=string,
            radius=distance
        )

        # Extend the business_list with the results of this search
        business_list.extend(response.get('results'))

        # Check if there are more results (next page) and retrieve them
        next_page_token = response.get('next_page_token')

        while next_page_token:
            # Wait for a few seconds before making the next request (rate limiting)
            time.sleep(3)

            # Perform additional requests to retrieve more results
            response = map_client.places_nearby(
                location=coordinates,
                keyword=string,
                radius=distance,
                page_token=next_page_token
            )

            # Extend the business_list with the additional results
            business_list.extend(response.get('results'))

            # Check if there are still more results
            next_page_token = response.get('next_page_token')

    # Convert the list of businesses to a Pandas DataFrame
    df = pd.DataFrame(business_list)

    # Save the DataFrame as an Excel file with a filename based on the store code
    df.to_excel(f'{store_code}_file.xlsx'.format(search_string), index=False)
